In [3]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [69]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
log_prices
target = pd.DataFrame(log_prices, columns=['PRICE'])
features.head()

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33


In [32]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1, 11))
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1, 11)

In [40]:
#type(features.mean())
#print(type(features.mean().values))
#features.mean().values.shape

In [49]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# using sklearn to find out MSE and RMSE
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [76]:
def get_log_estimate(nr_rooms, students_per_classroom,next_to_river=False,
                      high_confidence=True):
    
    # Configure Property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    
    
    # Make Prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [78]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

(2.776758191480403, 2.9642703266775334, 2.589246056283273, 68)

In [79]:
np.median(boston_dataset.target)

21.2

In [94]:
# converting the log price estimate using 1970s prices as well as the upper and the lower bouds to today's prices. Round the values to the  nearest 1000 dollars

ZILLON_MEDIAN_PRICE =  583.3
SCALE_FACTOR = ZILLON_MEDIAN_PRICE /np.median(boston_dataset.target)

SCALE_FACTOR

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15, 
                                next_to_river=False, high_confidence=False)

#Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 *SCALE_FACTOR
dollar_lo = np.e**lower * 1000 *SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_lo = np.around(dollar_lo, -3)

print(f'The estimated property value is {rounded_est}')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_lo} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0
At 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end.


In [113]:
def  get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    #docstring
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- True if the property is next to the rivver, False otherwise
    large)range -- True for a 95% prediction interval, False for 68% prediction interval
    
    """
    
    if rm <1 or ptratio < 1 :
        print('That is unrealistic. Try again.')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, ptratio, next_to_river=chas,
                                                   high_confidence=large_range)

    #Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 *SCALE_FACTOR
    dollar_lo = np.e**lower * 1000 *SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_lo = np.around(dollar_lo, -3)

    print(f'The estimated property value is {rounded_est}')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_lo} at the lower end to USD {rounded_hi} at the high end.')


In [112]:
get_dollar_estimate(rm=6, ptratio=-60, chas=True)

That is unrealistic. Try again.


In [115]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.
